# Auto Machine Learning with H20 Auto ML

Importing packages

In [2]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Data loading

In [6]:
xls = pd.ExcelFile('data/bank_term_deposit_marketing_analysis.xlsx')

In [7]:
xls.sheet_names

['PROCEDURE',
 'DATA DESCRIPTION',
 'Step 1 - Collect Information',
 'CLIENT_INFO',
 'LOAN_HISTORY',
 'MARKETING HISTORY',
 'SUBSCRIPTION HISTORY',
 'Step 2 - Merge Information',
 'CLIENT_MERGE',
 'Step 3 - Marketing Analysis',
 'DAILY RANGE',
 'JOB ANALYSIS',
 'Sheet3']

In [9]:
customer_info = pd.read_excel(xls, 'CLIENT_INFO')
loan_histo = pd.read_excel(xls, 'LOAN_HISTORY')
marketing_histo = pd.read_excel(xls, 'MARKETING HISTORY')
sub_histo = pd.read_excel(xls, 'SUBSCRIPTION HISTORY')

In [10]:
customer_info.head()

,ID,AGE,JOB,MARITAL,EDUCATION
0,2836,58,management,married,tertiary
1,2837,44,technician,single,secondary
2,2838,33,entrepreneur,married,secondary
3,2839,47,blue-collar,married,unknown
4,2840,33,unknown,single,unknown


In [11]:
loan_histo.head()

,ID,DEFAULT,BALANCE,HOUSING,LOAN
0,2836,no,2143,yes,no
1,2837,no,29,yes,no
2,2838,no,2,yes,yes
3,2839,no,1506,yes,no
4,2840,no,1,no,no


In [12]:
marketing_histo.head()

,ID,CONTACT,DAY,MONTH,DURATION,CAMPAIGN,PDAYS,PREVIOUS,POUTCOME
0,2836,unknown,5,may,261,1,-1,0,unknown
1,2837,unknown,5,may,151,1,-1,0,unknown
2,2838,unknown,5,may,76,1,-1,0,unknown
3,2839,unknown,5,may,92,1,-1,0,unknown
4,2840,unknown,5,may,198,1,-1,0,unknown


In [13]:
sub_histo.head()

,ID,TERM_DEPOSIT
0,2836,no
1,2837,no
2,2838,no
3,2839,no
4,2840,no


In [14]:
df = pd.merge(customer_info, loan_histo, on = 'ID')
df = pd.merge(df, marketing_histo, on = 'ID')
df = pd.merge(df, sub_histo, on = 'ID')

In [15]:
df.head()

,ID,AGE,JOB,MARITAL,EDUCATION,DEFAULT,BALANCE,HOUSING,LOAN,CONTACT,DAY,MONTH,DURATION,CAMPAIGN,PDAYS,PREVIOUS,POUTCOME,TERM_DEPOSIT
0,2836,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,2837,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,2838,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,2839,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,2840,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [16]:
df = df.drop('ID', axis=1)

Data prep & h2o cluster init

In [19]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 18.0.2+9-61, mixed mode, sharing)
  Starting server from E:\Python\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\valen\AppData\Local\Temp\tmpln9o0l9i
  JVM stdout: C:\Users\valen\AppData\Local\Temp\tmpln9o0l9i\h2o_valen_started_from_python.out
  JVM stderr: C:\Users\valen\AppData\Local\Temp\tmpln9o0l9i\h2o_valen_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.4
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_valen_xz6tyv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.988 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [22]:
h2o_df = h2o.H2OFrame(df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [23]:
h2o_df.describe()

Rows:45211
Cols:17




,AGE,JOB,MARITAL,EDUCATION,DEFAULT,BALANCE,HOUSING,LOAN,CONTACT,DAY,MONTH,DURATION,CAMPAIGN,PDAYS,PREVIOUS,POUTCOME,TERM_DEPOSIT
type,int,enum,enum,enum,enum,int,enum,enum,enum,int,enum,int,int,int,int,enum,enum
mins,18.0,,,,,-8019.0,,,,1.0,,0.0,1.0,-1.0,0.0,,
mean,40.93621021432809,,,,,1362.272057685082,,,,15.806418791886935,,258.1630797814693,2.7638406582468984,40.1978279622216,0.5803233726305563,,
maxs,95.0,,,,,102127.0,,,,31.0,,4918.0,63.0,871.0,275.0,,
sigma,10.61876204097539,,,,,3044.7658291685234,,,,8.32247615304459,,257.5278122651711,3.0980208832791796,100.1287459905981,2.303441044931215,,
zeros,0,,,,,3514,,,,0,,3,0,0,36954,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,58.0,management,married,tertiary,no,2143.0,yes,no,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown,no
1,44.0,technician,single,secondary,no,29.0,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,no
2,33.0,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,no


In [25]:
train, test = h2o_df.split_frame(ratios = [0.75])

x = train.columns
y = 'TERM_DEPOSIT'
x.remove(y)

Run H2o Auto ML

In [26]:
from h2o.automl import H2OAutoML

In [27]:
df.TERM_DEPOSIT.value_counts()

no     39922
yes     5289
Name: TERM_DEPOSIT, dtype: int64

Our output variable is slightly unbalanced.

In [28]:
aml = H2OAutoML(max_runtime_secs=600,
                balance_classes=True, #to address unbalanced output class
                stopping_metric='logloss',
                project_name='Final',
                seed = 64)

%time aml.train(x=x, y=y, training_frame = train)

AutoML progress: |
08:55:29.112: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████
09:05:21.372: GBM_lr_annealing_selection_AutoML_1_20220810_85529 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

█| (done) 100%
CPU times: total: 4.58 s
Wall time: 10min 3s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_4_AutoML_1_20220810_85529

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.03631409048341772
RMSE: 0.1905625631739291
LogLoss: 0.12593414129605632
Null degrees of freedom: 10059
Residual degrees of freedom: 10041
Null deviance: 7345.819244308889
Residual deviance: 2533.794922876654
AIC: 2571.794922876654
AUC: 0.9833990883775676
AUCPR: 0.8980179236819409
Gini: 0.9667981767551352

Confusion Matrix (Act/Pred) for max f1 @ thre

,,no,yes,Error,Rate
0,no,8583.0,279.0,0.0315,(279.0/8862.0)
1,yes,199.0,999.0,0.1661,(199.0/1198.0)
2,Total,8782.0,1278.0,0.0475,(478.0/10060.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.454630,0.806947,166.0
1,max f2,0.292330,0.862917,223.0
2,max f0point5,0.582588,0.840139,121.0
3,max accuracy,0.546218,0.954871,135.0
4,max precision,0.971080,1.000000,0.0
5,max recall,0.016217,1.000000,370.0
6,max specificity,0.971080,1.000000,0.0
7,max absolute_mcc,0.454630,0.780417,166.0
8,max min_per_class_accuracy,0.294002,0.933988,222.0
9,max mean_per_class_accuracy,0.254334,0.938306,237.0



Gains/Lift Table: Avg response rate: 11,91 %, avg score: 12,85 %


...


Gains/Lift Table: Avg response rate: 11,85 %, avg score: 11,85 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010011,0.817888,7.067514,7.067514,0.837758,0.857504,0.837758,0.857504,0.070752,0.070752,606.751444,606.751444,0.068910
1,2,0.020022,0.756492,6.520031,6.793773,0.772861,0.785169,0.805310,0.821337,0.065272,0.136024,552.003092,579.377268,0.131602
2,3,0.030003,0.707987,6.015177,6.534752,0.713018,0.731914,0.774606,0.791588,0.060040,0.196064,501.517691,553.475165,0.188392
3,4,0.040014,0.668954,5.499721,6.275803,0.651917,0.688476,0.743911,0.765791,0.055057,0.251121,449.972074,527.580296,0.239496
4,5,0.050025,0.636785,5.151322,6.050774,0.610619,0.652651,0.717237,0.743150,0.051570,0.302691,415.132214,505.077403,0.286643
5,6,0.100021,0.486449,4.664090,5.357637,0.552865,0.559864,0.635075,0.651534,0.233184,0.535874,366.409033,435.763689,0.494466
6,7,0.150016,0.330725,3.597728,4.771116,0.426462,0.411511,0.565551,0.571542,0.179870,0.715745,259.772780,377.111601,0.641806
7,8,0.200012,0.202146,2.381875,4.173894,0.282339,0.263392,0.494759,0.494516,0.119083,0.834828,138.187519,317.389399,0.720184
8,9,0.300003,0.065240,1.136125,3.161404,0.134672,0.120169,0.374742,0.369746,0.113602,0.948430,13.612457,216.140386,0.735626
9,10,0.399994,0.024834,0.331370,2.453948,0.039279,0.041032,0.290882,0.287573,0.033134,0.981565,-66.863033,145.394754,0.659778




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.899784,0.002765,0.898979,0.897421,0.903235,0.897155,0.902132
1,auc,0.935739,0.002518,0.935619,0.932496,0.939147,0.936984,0.934452
2,err,0.100216,0.002765,0.101021,0.102579,0.096765,0.102845,0.097868
3,err_count,678.600000,14.046351,683.000000,688.000000,667.000000,694.000000,661.000000
4,f0point5,0.587170,0.012099,0.590413,0.567959,0.601298,0.589618,0.586563
5,f1,0.635734,0.007549,0.639578,0.627706,0.640043,0.643737,0.627606
6,f2,0.693384,0.013711,0.697674,0.701500,0.684126,0.708795,0.674824
7,lift_top_group,7.000369,0.177826,7.067078,6.730817,7.006186,6.976542,7.221226
8,logloss,0.196040,0.001961,0.197996,0.195316,0.193236,0.197807,0.195843
9,max_per_class_error,0.261738,0.023658,0.257353,0.238845,0.282950,0.240000,0.289541



See the whole table with table.as_data_frame()


Models leaderboard exploration & ensemble exploration

In [29]:
leaderboard = aml.leaderboard
leaderboard.head(rows = leaderboard.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_4_AutoML_1_20220810_85529,0.935786,0.196029,0.63451,0.166893,0.248892,0.0619471
StackedEnsemble_AllModels_3_AutoML_1_20220810_85529,0.935755,0.196003,0.634941,0.167477,0.248878,0.0619402
StackedEnsemble_AllModels_2_AutoML_1_20220810_85529,0.935141,0.196862,0.629847,0.181006,0.249535,0.0622678
StackedEnsemble_AllModels_1_AutoML_1_20220810_85529,0.934917,0.197271,0.629602,0.176777,0.249648,0.062324
StackedEnsemble_BestOfFamily_4_AutoML_1_20220810_85529,0.934444,0.197964,0.624177,0.158916,0.250256,0.0626278
StackedEnsemble_BestOfFamily_3_AutoML_1_20220810_85529,0.934306,0.197947,0.625602,0.166936,0.250214,0.0626072
StackedEnsemble_BestOfFamily_1_AutoML_1_20220810_85529,0.934004,0.199444,0.626638,0.169035,0.2509,0.0629508
StackedEnsemble_BestOfFamily_2_AutoML_1_20220810_85529,0.933844,0.198649,0.625723,0.165438,0.250463,0.0627318
GBM_grid_1_AutoML_1_20220810_85529_model_16,0.933839,0.205577,0.621605,0.165836,0.255408,0.0652333
GBM_1_AutoML_1_20220810_85529,0.933791,0.206849,0.6257,0.169443,0.255931,0.0655005
